In [1]:
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squred_error
from sklearn.svm import SVC()

In [2]:
df_user = pd.read_csv('./data/train_user.csv')

In [3]:
df_user = df_user.drop(['Unnamed: 0','month_freq',
                        'month_rate', 'dow_freq', 'dow_rate', 
                        'hour_freq', 'hour_rate'], axis=1)

In [4]:
df_user.head()

,fullVisitorId,is_transaction,transactionRevenue_total,bounces_mean,hits_mean,pageviews_mean,visitNumber_total,is_source_googleplex_freq,is_source_googleplex_rate,is_source_direct_freq,...,h_13_17_freq,h_13_17_rate,h_18_23_freq,h_18_23_rate,browser_chrome_freq,browser_chrome_rate,is_mobile_freq,is_mobile_rate,system_google_freq,system_google_rate
0,4823595352351,0,0.0,1.0,1.0,1.0,1,0,0.0,0,...,1,1.0,0,0.0,1,1.0,0,0.0,0,0.0
1,5103959234087,0,0.0,0.0,10.0,8.0,1,0,0.0,0,...,0,0.0,1,1.0,1,1.0,1,1.0,1,1.0
2,10278554503158,0,0.0,0.0,11.0,8.0,1,0,0.0,0,...,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0
3,20424342248747,0,0.0,0.0,17.0,13.0,1,0,0.0,0,...,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0
4,26722803385797,0,0.0,0.0,3.0,2.0,1,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


## split

In [ ]:
X = df_trans.drop(['is_transaction', 'fullVisitorId','transactionRevenue_total'], axis=1)
y = df_trans['transactionRevenue_total']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Stacking

In [133]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np

class StackedRegressor(BaseEstimator, ClassifierMixin):  
    def __init__(self, classifier, regressor):
        self.classifier = classifier
        self.regressor = regressor
#         args, _, _, values = inspect.getargvalues(inspect.currentframe())
#         values.pop("self")
#         for arg, val in values.items():
#             for param, value in val.get_params().items():
#                 arg_final = arg + '_' + ''
#                 setattr(self, )
        
    def fit(self, X, y):
        class_labels = pd.Series(np.where(y>0,1,0))
        self.classifier.fit(X,class_labels)
        
        X = X.reset_index(drop=True)
        class_labels = class_labels.reset_index(drop=True)
        y = y.reset_index(drop=True)    
        X_reg = X[class_labels==1]
        y = y[class_labels==1]
        self.regressor.fit(X_reg,y)
        
    def predict(self, X):
        class_predict = self.classifier.predict(X)
        regressor_predict = self.regressor.predict(X)
        np.multiply(class_predict,regressor_predict)
        return np.multiply(class_predict,regressor_predict)
    
    def score(self, X, y):
        return np.sqrt(np.mean((y - self.predict(X))**2))
mse_scorer = make_scorer(mean_squared_error)

In [ ]:
## Grid search

In [ ]:
svc = []
for C in [0.8, 1]:
    for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
        svc.append(sklearn.svm.SVC(C=1.0, kernel=’rbf’, random_state=42))

In [158]:
rfc = []
for n in [30, 50]:
    for min_l in [3, 5]:
        rfc.append(RandomForestClassifier(random_state=42, n_estimators = n, min_samples_leaf = min_l))

In [159]:
logic = []
for penalty in [ 'l1', 'l2']:
    for C in [0.5, 0.7, 1]:
        logic.append(LogisticRegression(penalty=penalty, C=C, random_state=42))

In [160]:
lr = []
for normalize in [True, False]:
    lr.append(LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None))

In [161]:
rfr = []
for n in [30, 50]:
    for min_l in [3, 5]:
        rfr.append(RandomForestRegressor(random_state=42, n_estimators = n, min_samples_leaf = min_l))

In [162]:
parameters = {'classifier': rfc + logic + svc, 
              'regressor': lr + rfr}

In [164]:
gs = GridSearchCV(cv=5, estimator=StackedRegressor(RandomForestClassifier(random_state=42),
                                                   LinearRegression()),param_grid=parameters,
                 scoring = mse_scorer)

In [ ]:
gs.fit(X=X_train, y=y_train)

In [118]:
gs.best_estimator_

StackedRegressor(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
         regressor=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False))

In [119]:
gs.best_score_ # best mse

-2.0282318957517735

In [ ]:
gs.score(self, X_test, y_test) # rmse